# Introduction
In notebook 24, I've looked at the formation of the NMPs. To systematically go through the majority of the network, I will not tackle the 'stage' of the NDPs, and then from there move up towards the NTPs. Hopefully this will solve quite some issues that we have. 

In [139]:
import cameo
import pandas as pd
import cobra.io
import escher
from escher import Builder
from cobra import Reaction, Metabolite

In [188]:
model = cobra.io.read_sbml_model('../model/g-thermo.xml')

In [3]:
model_e_coli = cameo.load_model ('iML1515')

In [4]:
model_b_sub = cameo.load_model('iYO844')

__CMP & CDP node__ Mostly had to change the reversability of these reactions, as they were wrong. The rest seemed fine.

In [ ]:
#bounds in the wrong direction
model.reactions.RNDR3.bounds = (-1000,0)

In [ ]:
#I can remove this one reaction involving RNA
model.remove_reactions(model.reactions.PRPCT)

In [ ]:
#also in the wrong direction
model.reactions.PYK4.bounds = (-1000,0)

In [ ]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

In [ ]:
#fixing CMP node, aside from this it seems fine. 

__GDP__

In [ ]:
#can also remove this one reaction involving RNA
model.remove_reactions(model.reactions.PRPGT)

In [ ]:
#rename
model.reactions.ATGDm.id = 'NDPK1'

In [ ]:
#rename
model.reactions.GTPOPm.id = 'PYK3'

In [ ]:
#switch reaction direction, see delta g
model.reactions.PYK3.bounds = (-1000,0)

In [ ]:
#switch directionality
model.reactions.RNDR2.bounds = (-1000,0)

In [ ]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

In [ ]:
#check gmp node: seems fine

__XMP to other mets__
The model doesn't contain XDP, so instead I will just check that the XMP that is formed is properly converted furhter. After inspection, it is, and so we don't have to change anything here. 

__AMP & ADP__ There are 108 reactions involving ADP. This is a lot to go through in this notebook. Instead, here I will go through the ADP reactions that do not contain ATP, to check that those are correct. 

I've checked the reversable ATP reactions in another notebook already, so for now we can assume they are fine, otherwise I'll look at them again at a later stage.

In [5]:
model = cobra.io.read_sbml_model('../model/g-thermo.xml')

In [6]:
#rename
model.reactions.MCS.id = 'MECDPS'

In [7]:
for rct in model.metabolites.adp_c.reactions:
    if rct in model.metabolites.atp_c.reactions:
        continue
    else:
        print (rct.id, ':',rct.reaction)

RNDR1 : dadp_c + h2o_c + trdox_c --> adp_c + h_c + trdrd_c
PRPAT : pi_c <=> RNA_c + adp_c


So there are only two reactions with ADP but no ATP: RNDR1 and PRPAT. There should also be more, e.g. converting ADP to dADP, and ADP back to AMP. So I will look into this a bit more. 

In [8]:
#removing PRPAT, as we know it is wrong
model.remove_reactions(model.reactions.PRPAT)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\model.py:716: UserWarning:

need to pass in a list

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\group.py:110: UserWarning:

need to pass in a list



In [9]:
#change direction of reaction
model.reactions.RNDR1.bounds = (-1000,0)

In [10]:
#also for the amp node, just check which reactions do not involve ATP
for rct in model.metabolites.amp_c.reactions:
    if rct in model.metabolites.atp_c.reactions:
        continue
    else:
        print (rct.id, ':',rct.reaction)

PAMPT : coa_c + pamp_c --> amp_c + h_c + ppcoa_c
NTD7 : amp_c + h2o_c --> adn_c + pi_c
DNADDP : dnad_c + h2o_c --> amp_c + h_c + nicrnt_c
BTN5AMPL : b5amp_c + h2o_c <=> amp_c + btn_c + 2.0 h_c
ADPRIBPH : adprib_c + h2o_c --> amp_c + h_c + r5p_c
NADDP : h2o_c + nad_c --> amp_c + h_c + nmn_c
FADDP : fad_c + h2o_c --> amp_c + fmn_c + h_c
DPCOAPP : dpcoa_c + h2o_c --> amp_c + h_c + pan4p_c
ADPT : amp_c + 2.0 h_c + ppi_c --> ade_c + prpp_c
ADSL1r : dcamp_c --> amp_c + fum_c + h_c


In [ ]:
#rct DPCOAPP should be Ec 2.7.7.3, not EC3.6.1.9. So here I will change the reaction to the correct.

In [11]:
model.reactions.DPCOAPP.id = 'PTPATi'

In [12]:
model.reactions.PTPATi.name = 'Pantetheine-phosphate adenylyltransferase'
model.reactions.PTPATi.annotation = model_e_coli.reactions.PTPATi.annotation
model.reactions.PTPATi.bounds = (-1000,0)
model.reactions.PTPATi.notes = {}

In [13]:
model.reactions.PTPATi.add_metabolites({model.metabolites.atp_c:1,model.metabolites.amp_c:-1, model.metabolites.ppi_c:-1, model.metabolites.h2o_c:1})

In [25]:
model.reactions.NNATm.id = 'NNATr'

In [31]:
model.reactions.ADPRIBPH.id = 'ADPRDP'

In [35]:
#change directionality
model.reactions.ADPT.bounds = (-1000,0)

In [40]:
#make reversible
model.reactions.ADSL1r.bounds = (-1000,1000)

In [42]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

__UMP and UDP__

In [47]:
#make irreversible
model.reactions.UPPRT.bounds = (-1000,0)

In [55]:
#switch directionality
#cannot be done as it will kill biomass for now
#model.reactions.OMPDC.bounds = (-1000,1000)

In [63]:
#DTPCUNPPT should be the same as e. coli ACGAMT
#rename the Undecaprenyl phosphate metabolite
model.metabolites.dtpcunp_c.id = 'udcpp_c'

In [65]:
model.reactions.DTPCUNPPT.id = 'ACGAMT'

In [ ]:
# DTPCUNPT should be PAPPT3

In [71]:
model.reactions.DTPCUNPT.id = 'PAPPT3'

In [72]:
model.reactions.PAPPT3.bounds = (0,1000)

In [82]:
#should be reversible
model.reactions.NDPK2.bounds = (-1000,1000)

In [84]:
#wrong directionality
model.reactions.PYK2.bounds = (-1000,0)

In [96]:
model.reactions.PYK5.bounds = (-1000,0)

In [114]:
#UACGAMT should be the same as UAGPT3
model.reactions.UACGAMT.id = 'UAGPT3'

In [115]:
model.reactions.UAGPT3.annotation = model_e_coli.reactions.UAGPT3.annotation

In [116]:
model.metabolites.uddaalagmdaa_c.id = 'uaagmda_c'

Reaction identifier,UAGPT3
Name,R05032
Memory address,0x02252aae9f88
Stoichiometry,uacgam_c + uagmda_c <=> uddaalagmdaa_c + udp_c UDP-N-acetyl-D-glucosamine + Undecaprenyl-diphospho-N-acetylmuramoyl-L-alanyl-D-glutamyl-meso- <=> Undecaprenyl-diphospho-N-acetylmuramoyl-(N-acetylglucosamine)-L- + UDP
GPR,RTMO00339
Lower bound,-1000.0
Upper bound,1000.0


In [118]:
model.metabolites.uaagmda_c.notes = {}
model.metabolites.uaagmda_c.annotation = model_e_coli.metabolites.uaagmda_c.annotation 

In [144]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

__dCMP and dCDP__ These nodes seem to be correct.

In [149]:
model = cobra.io.read_sbml_model('../model/g-thermo.xml')

__dGMP and dGDP__

In [161]:
#should be in reverse direction
model.reactions.AGPOP.bounds = (-1000,0)

__dTMP and dTDP__ Seems to be fine

__dUMP and dUDP__ is missing one reaction

In [171]:
model.add_reaction(Reaction(id='RNDR4'))

In [173]:
model.reactions.RNDR4.name = 'Ribonucleoside-diphosphate reductase (UDP)'
model.reactions.RNDR4.bounds = (0,1000)
model.reactions.RNDR4.annotation = model_e_coli.reactions.RNDR4.annotation

In [174]:
model.reactions.RNDR4.add_metabolites({model.metabolites.trdrd_c:-1, model.metabolites.udp_c:-1, model.metabolites.dudp_c:1, model.metabolites.h2o_c:1, model.metabolites.trdox_c:1})

In [178]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

__dAMP and dADP__ Both nodes seem fine here.

In [187]:
rct_id = []
rct_rct =[]
for rct in model.metabolites.damp_c.reactions:
    rct_id.append(rct.id)
    rct_rct.append(rct.reaction)
    
rct_df = pd.DataFrame({'ID':rct_id, 'reaction':rct_rct})
rct_df

,ID,reaction
0,NTD6,damp_c + h2o_c --> dad_2_c + h_c + pi_c
1,biomass,0.029903042 accoa_c + 0.54554044 ala__L_c + 0....
2,DADNK,atp_c + dad_2_c --> adp_c + damp_c
3,DADK,atp_c + damp_c <=> adp_c + dadp_c


In [182]:
rct_id = []
rct_rct =[]
for rct in model_e_coli.metabolites.dadp_c.reactions:
    rct_id.append(rct.id)
    rct_rct.append(rct.reaction)
    
rct_df = pd.DataFrame({'ID':rct_id, 'reaction':rct_rct})
rct_df

,ID,reaction
0,NDPK8,atp_c + dadp_c <=> adp_c + datp_c
1,DADK,atp_c + damp_c <=> adp_c + dadp_c
2,RNDR1,adp_c + trdrd_c --> dadp_c + h2o_c + trdox_c
3,RNDR1b,adp_c + grxrd_c --> dadp_c + grxox_c + h2o_c


In [189]:
model.optimize().objective_value

10.7599977886498

In [185]:
#check if this is the cause of ump related issues
#switch directionality, cannot bew done now, noted in issue
#model.reactions.OMPDC.bounds = (-1000,0)